In [1]:
import requests
import numpy as np
import pandas as pd
import datetime
from geopy.geocoders import Nominatim
from collections import defaultdict

url = "https://covid-api.com/api/provinces/USA"

response = requests.get(url)

res = response.json()
res['data']

[{'iso': 'USA',
  'name': 'US',
  'province': ' Norfolk County, MA',
  'lat': '42.1767',
  'long': '-71.1449'},
 {'iso': 'USA',
  'name': 'US',
  'province': 'Alabama',
  'lat': '32.3182',
  'long': '-86.9023'},
 {'iso': 'USA',
  'name': 'US',
  'province': 'Alameda County, CA',
  'lat': '37.6017',
  'long': '-121.7195'},
 {'iso': 'USA',
  'name': 'US',
  'province': 'Alaska',
  'lat': '61.3707',
  'long': '-152.4044'},
 {'iso': 'USA',
  'name': 'US',
  'province': 'American Samoa',
  'lat': '-14.270999999999999',
  'long': '-170.132'},
 {'iso': 'USA',
  'name': 'US',
  'province': 'Arizona',
  'lat': '33.7298',
  'long': '-111.4312'},
 {'iso': 'USA',
  'name': 'US',
  'province': 'Arkansas',
  'lat': '34.9697',
  'long': '-92.3731'},
 {'iso': 'USA',
  'name': 'US',
  'province': 'Ashland, NE',
  'lat': None,
  'long': None},
 {'iso': 'USA',
  'name': 'US',
  'province': 'Bennington County, VT',
  'lat': '43.0279',
  'long': '-73.1350'},
 {'iso': 'USA',
  'name': 'US',
  'province': 'B

In [2]:
geolocator = Nominatim(user_agent="my_geopy_app")


In [3]:
def get_state(lat,lon):
    location = geolocator.reverse(str(lat)+","+str(lon))
    return location.raw['address']['state']

get_state(32.3182,-86.9023)

'Alabama'

In [4]:

#url = "https://covid-api.com/api/reports/total?date=2020-03-14&iso=USA"

#statistics for the US

stats = pd.DataFrame(columns = range(11))
stats.columns = ['date', 'last_update', 'confirmed', 'confirmed_diff', 'deaths', 'deaths_diff', 'recovered', 'recovered_diff', 'active', 'active_diff', 'fatality_rate']

def get_date_range(start_date, end_date):
  if start_date > end_date:
    return []
  
  delta = end_date - start_date
  dates = [start_date + datetime.timedelta(days=i) for i in range(delta.days + 1)]
  return dates

dates = get_date_range(datetime.date(2020, 3, 14), datetime.date(2022, 3, 14))


def get_data(date, iso = "USA"):
    url = f"https://covid-api.com/api/reports/total?date={date}&iso={iso}"
    response = requests.get(url)
    res = response.json()
    return res['data']

for i,date in enumerate(dates):
    data = get_data(date)
    stats.loc[i] = data.values()

print(stats.shape)     

KeyboardInterrupt: 

In [ ]:
stats.to_csv('../dataset/US_stats.csv', index=False)

In [ ]:
stats.loc[1]

date                       2020-03-15
last_update       2020-03-15 18:33:03
confirmed                        3499
confirmed_diff                    769
deaths                             63
deaths_diff                         9
recovered                          12
recovered_diff                      0
active                           3424
active_diff                       760
fatality_rate                   0.018
Name: 1, dtype: object

In [ ]:
#get all regions and cities in the US

url = "https://covid-api.com/api/provinces/USA?order=name"
response = requests.get(url)
res = response.json()
print(res['data'][0].keys())
print(len(res['data']))

#keep only provices with valid lat and long
new_res = []
for i in range(len(res['data'])):
    if res['data'][i]['lat'] and res['data'][i]['lat'] != '0.0':
        new_res.append(res['data'][i])

for i in range(len(new_res)):
    print(new_res[i]['province'], new_res[i]['lat'], new_res[i]['long'])
print(len(new_res))    

dict_keys(['iso', 'name', 'province', 'lat', 'long'])
202
Montgomery County, PA 40.2290 -75.3879
Fairfax County, VA 38.9085 -77.2405
Rockingham County, NH 42.9931 -71.0498
Washington, D.C. 38.9072 -77.0369
Berkshire County, MA 42.3118 -73.1822
Davidson County, TN 36.1343 -86.8220
Douglas County, OR 43.1261 -123.2492
Fresno County, CA 36.9859 -119.2321
Harford County, MD 39.5839 -76.3637
Hendricks County, IN 39.8065 -86.5401
Hudson County, NJ 40.7453 -74.0535
Johnson County, KS 38.8454 -94.8521
Kittitas County, WA 47.1750 -120.9319
Manatee County, FL 27.4799 -82.3452
Marion County, OR 44.8446 -122.5927
Okaloosa County, FL 30.5773 -86.6611
Polk County, GA 34.0132 -85.1479
Riverside County, CA 33.9533 -117.3961
Shelby County, TN 35.1269 -89.9253
Spokane County, WA 47.6587 -117.4225
St. Louis County, MO 38.6103 -90.4125
Suffolk County, NY 40.9849 -72.6151
Ulster County, NY 41.8586 -74.3118
Unassigned Location, VT 44.3378 -72.7563
Unknown Location, MA 42.4072 -71.3824
Volusia County, FL 29.

In [29]:
#map each lat long to a state

# a dict for all the states - lat long

states = defaultdict(list)

def get_state(lat,lon):
    location = geolocator.reverse(str(lat)+","+str(lon))
    try:
        if location.raw['address']['state']:
            return location.raw['address']['state']
    except:
        return None

for i in range(len(new_res)):
    lat,lon = new_res[i]['lat'],new_res[i]['long']
    state = get_state(lat,lon)
    print(state)
    states[state].append(new_res[i]['province'])

Pennsylvania
Virginia
New Hampshire
District of Columbia
Massachusetts
Tennessee
Oregon
California
Maryland
Indiana
New Jersey
Kansas
Washington
Florida
Oregon
Florida
Georgia
California
Tennessee
Washington
Missouri
New York
New York
Vermont
Massachusetts
Florida
Washington
Iowa
Kentucky
Vermont
Minnesota
Florida
Georgia
Texas
Kentucky
Louisiana
California
South Carolina
New York
Massachusetts
Illinois
None
California
California
Georgia
Arizona
Colorado
Florida
New Jersey
Oregon
Texas
Pennsylvania
Iowa
Maryland
North Carolina
South Carolina
Tennessee
Virginia
Indiana
Kentucky
District of Columbia
Nevada
New Hampshire
Illinois
Minnesota
Nebraska
Ohio
Massachusetts
Rhode Island
California
Wisconsin
California
California
Connecticut
Washington
None
Arizona
Oklahoma
California
Utah
Wisconsin
Kansas
Louisiana
California
Missouri
Texas
Vermont
Alaska
Arkansas
Delaware
Idaho
Maine
Michigan
Mississippi
Montana
California
New Mexico
California
North Dakota
Nebraska
South Dakota
California
West

In [27]:
print(i)
print(res['data'][i+1]['lat'])
print(res['data'][i+1]['long'])

41
35.4437
139.6380


In [ ]:
states

defaultdict(list,
            {'Pennsylvania': ['Montgomery County, PA',
              'Pennsylvania',
              'Delaware County, PA',
              'Wayne County, PA'],
             'Virginia': ['Fairfax County, VA', 'Virginia'],
             'New Hampshire': ['Rockingham County, NH',
              'New Hampshire',
              'Grafton County, NH'],
             'District of Columbia': ['Washington, D.C.',
              'District of Columbia'],
             'Massachusetts': ['Berkshire County, MA',
              'Unknown Location, MA',
              'Massachusetts',
              'Boston, MA',
              ' Norfolk County, MA',
              'Suffolk County, MA',
              'Middlesex County, MA',
              'Norwell County, MA',
              'Plymouth County, MA',
              'Norfolk County, MA'],
             'Tennessee': ['Davidson County, TN',
              'Shelby County, TN',
              'Tennessee',
              'Williamson County, TN'],
             'Oreg

In [37]:
print(list(states.keys()))


['Pennsylvania', 'Virginia', 'New Hampshire', 'District of Columbia', 'Massachusetts', 'Tennessee', 'Oregon', 'California', 'Maryland', 'Indiana', 'New Jersey', 'Kansas', 'Washington', 'Florida', 'Georgia', 'Missouri', 'New York', 'Vermont', 'Iowa', 'Kentucky', 'Minnesota', 'Texas', 'Louisiana', 'South Carolina', 'Illinois', None, 'Arizona', 'Colorado', 'North Carolina', 'Nevada', 'Nebraska', 'Ohio', 'Rhode Island', 'Wisconsin', 'Connecticut', 'Oklahoma', 'Utah', 'Alaska', 'Arkansas', 'Delaware', 'Idaho', 'Maine', 'Michigan', 'Mississippi', 'Montana', 'New Mexico', 'North Dakota', 'South Dakota', 'West Virginia', 'Wyoming', 'United States Virgin Islands', 'Alabama', 'Puerto Rico', 'Guam', 'Northern Mariana Islands', 'Hawaii']


In [ ]:
#get the keys from the state dictionary
#get total stats for each state (use try catch)

def get_total_stats(date,state):
    url = 'https://covid-api.com/api/reports?date='+date+'&iso=USA&region_name=US&region_province='+state
    try:
        response = requests.get(url)
        res = response.json()
        return res['data'][0]
    except:
        return None

get_total_stats('2020-04-16','Guam')

{'date': '2020-04-16',
 'confirmed': 135,
 'deaths': 5,
 'recovered': 0,
 'confirmed_diff': 0,
 'deaths_diff': 0,
 'recovered_diff': 0,
 'last_update': '2020-04-16 23:30:51',
 'active': 130,
 'active_diff': 0,
 'fatality_rate': 0.037,
 'region': {'iso': 'USA',
  'name': 'US',
  'province': 'Guam',
  'lat': '13.4443',
  'long': '144.7937',
  'cities': []}}